In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import trimesh

from magnet_pinn.generator.io import MeshWriter
from magnet_pinn.generator.phantoms import Tissue
from magnet_pinn.generator.properties import PropertySampler
from magnet_pinn.generator.transforms import ToMesh, MeshesCutout, MeshesCleaning, Compose

First generate 3d objects we suppose to use

In [3]:
tissue = Tissue(
    num_children_blobs=3,
    initial_blob_radius=100,
    initial_blob_center_extent={
        "x": [-5, 5],
        "y": [-5, 5],
        "z": [-50, 50],
    },
    blob_radius_decrease_per_level=0.3,
    num_tubes=10,
    relative_tube_max_radius=0.1,
    relative_tube_min_radius=0.01
)
raw_3d_stgructures = tissue.generate(seed=42)
raw_3d_stgructures

PhantomItem(parent=Blob(relative_disruption_strength=0.1, empirical_max_offset=np.float64(0.14122109965674973), empirical_min_offset=np.float64(-0.14253592429194548)), children=[Blob(relative_disruption_strength=0.1, empirical_max_offset=np.float64(0.13623293210239198), empirical_min_offset=np.float64(-0.16040753555150875)), Blob(relative_disruption_strength=0.1, empirical_max_offset=np.float64(0.14492809120125774), empirical_min_offset=np.float64(-0.14306291053726253)), Blob(relative_disruption_strength=0.1, empirical_max_offset=np.float64(0.14863081342064496), empirical_min_offset=np.float64(-0.135859752111401))], tubes=[Tube(direction=array([ 0.98738769,  0.10157726, -0.12143971]), height=10000), Tube(direction=array([-0.53035753,  0.84641048,  0.04806434]), height=10000), Tube(direction=array([-0.21301774,  0.72321758,  0.65694731]), height=10000), Tube(direction=array([-0.15008846,  0.13319334, -0.97965963]), height=10000), Tube(direction=array([0.54344636, 0.61938477, 0.56659382]

Process 3d obejcts into the meshes

In [4]:
workflow = Compose([
    ToMesh(),
    MeshesCutout(),
    MeshesCleaning()
])
meshes = workflow(raw_3d_stgructures)
meshes

PhantomItem(parent=<trimesh.Trimesh(vertices.shape=(42361, 3), faces.shape=(84762, 3))>, children=[<trimesh.Trimesh(vertices.shape=(10355, 3), faces.shape=(20710, 3))>, <trimesh.Trimesh(vertices.shape=(10249, 3), faces.shape=(20494, 3))>, <trimesh.Trimesh(vertices.shape=(10222, 3), faces.shape=(20440, 3))>], tubes=[<trimesh.Trimesh(vertices.shape=(220, 3), faces.shape=(436, 3))>, <trimesh.Trimesh(vertices.shape=(102, 3), faces.shape=(200, 3))>, <trimesh.Trimesh(vertices.shape=(141, 3), faces.shape=(278, 3))>, <trimesh.Trimesh(vertices.shape=(202, 3), faces.shape=(400, 3))>, <trimesh.Trimesh(vertices.shape=(133, 3), faces.shape=(262, 3))>, <trimesh.Trimesh(vertices.shape=(203, 3), faces.shape=(402, 3))>, <trimesh.Trimesh(vertices.shape=(96, 3), faces.shape=(188, 3))>, <trimesh.Trimesh(vertices.shape=(104, 3), faces.shape=(204, 3))>, <trimesh.Trimesh(vertices.shape=(131, 3), faces.shape=(258, 3))>, <trimesh.Trimesh(vertices.shape=(202, 3), faces.shape=(400, 3))>])

In [5]:
trimesh.boolean.union(
    meshes.children + meshes.tubes,
    engine='manifold',
).show()

Sample properties for each mesh

In [6]:
prop_sampler = PropertySampler(
    {
        "density": {
            "min": 400,
            "max": 2000
        },
        "conductivity": {
            "min": 0,
            "max": 2.5
        },
        "permittivity": {
            "min": 1,
            "max": 71
        }
    }
)
prop = prop_sampler.sample_like(meshes)
prop

PhantomItem(parent=PropertyItem(conductivity=0.5893124794391222, permittivity=40.94894169803723, density=425.12504355777344), children=[PropertyItem(conductivity=0.274120145967571, permittivity=69.35627591845905, density=1763.746858988515), PropertyItem(conductivity=2.291168344796243, permittivity=14.409632444878312, density=1790.970615720967), PropertyItem(conductivity=1.7450782364013073, permittivity=7.017226346113281, density=1025.408289735305)], tubes=[PropertyItem(conductivity=0.34168987686428093, permittivity=37.55549960758304, density=1837.7679206272064), PropertyItem(conductivity=1.0688032666195735, permittivity=32.59234330313197, density=447.88772386843794), PropertyItem(conductivity=2.363680972123779, permittivity=67.56381094268316, density=1949.2154151471289), PropertyItem(conductivity=0.30617750217260514, permittivity=19.61174374183398, density=699.7829772683458), PropertyItem(conductivity=0.3177261278132862, permittivity=5.889251962912194, density=1122.7361425397087), Prop

Now export meshes as `.stl` files and also pandas dataframe with columns `file`, `permittivity`, `conductivity`, `density` into the `materials.txt` file in the `.csv` format.

In [7]:
writer = MeshWriter("../gen_data/raw/tissue_meshes")
writer.write(meshes, prop)